In [1]:
import pandas as pd
from utils.DataBaseBuffer import DataBaseBuffer
import numpy as np

In [2]:
def transform_data(data_buffer: DataBaseBuffer, data: pd.DataFrame,tags:np.array, append:bool):
    pivot_df = data.pivot(index='time', columns='asset', values='value')  
    assets = pivot_df.columns.values
    elementos_excluidos = np.setdiff1d(tags, assets)
    nuevas_columnas = {col: [np.nan] * len(pivot_df) for col in elementos_excluidos}
    pivot_df = pivot_df.assign(**nuevas_columnas)

    pivot_df = pivot_df[tags]
        
    if not append:
        pivot_df.to_csv(f"data/n_{data_buffer.process_name}_p{data_buffer.process}_e{data_buffer.exec_id}.csv", header = True)
    else:
        pivot_df.to_csv(f"data/n_{data_buffer.process_name}_p{data_buffer.process}_e{data_buffer.exec_id}.csv", mode='a', header = False)
    
    return pivot_df.isna().sum().sum()

In [3]:
def extract_data_of_process(process:int):
    data_buffer = DataBaseBuffer(process, batch_size= 100000)
    while True:
        tags = data_buffer.tags_execution()
        total_nan = 0
        data_buffer.restart()
        data_temp = pd.DataFrame([], columns =["id","time","value","asset","execution"])
        while data_buffer.next():
            data = data_buffer.current_data
            data = pd.concat([data_temp, data], ignore_index=True)
            last_time = data['time'].iloc[-1]
            data_temp = data[data["time"]==last_time]
            data = data.drop(data_temp.index)

            nans_exec = transform_data(data_buffer, data, tags, data_buffer.current_batch != 0)
            total_nan = total_nan + nans_exec

        nans_exec = transform_data(data_buffer, data_temp, tags, True)
        total_nan = total_nan + nans_exec
        print(f"total datos nans {total_nan} for exec {data_buffer.exec_id}")
        
        if not data_buffer.next_execution():
            break

In [4]:
for i in range(2,9):
    print(f"{i}")
    extract_data_of_process(i)

2
total datos nans 0.0 for exec 1
total datos nans 0 for exec 4
3
total datos nans 0 for exec 2
total datos nans 0 for exec 3
total datos nans 0 for exec 5
total datos nans 0 for exec 6
total datos nans 0 for exec 7
4
total datos nans 0 for exec 8
total datos nans 0.0 for exec 9
total datos nans 0.0 for exec 10
total datos nans 0 for exec 11
5
total datos nans 0.0 for exec 12
total datos nans 0.0 for exec 13
total datos nans 0.0 for exec 14
total datos nans 0.0 for exec 15
total datos nans 0.0 for exec 16
total datos nans 0.0 for exec 17
total datos nans 0 for exec 18
total datos nans 2 for exec 19
6
total datos nans 20 for exec 20
7
total datos nans 0 for exec 21
8
total datos nans 0 for exec 22
total datos nans 3 for exec 23
